# MCP Medical Prediction System for eICU Data



This notebook demonstrates how to build a medical prediction system using IntelliNode, where:

- **MCP Server**: Serves patient data from eICU dataset via HTTP.
- **Context Agent**: Loads patient data from MCP server.
- **Prediction Agent**: Predicts patient mortality using clinical data and few-shot learning.

The provided data is a sample of 10 patients from physionet (eICU Collaborative Research Database).


**Objectives**

In this tutorial, you will learn how to:

- Set up MCP DataFrame server for serving medical data over HTTP.
- Create MCP client agents to query structured data.
- Build a 2-agent flow for data loading and prediction.

## Setup and Imports

In [5]:
!pip install polars

In [6]:
!pip install "intelli[mcp]" python-dotenv

In [8]:
import os
import asyncio
import json
import pandas as pd
import re
from dotenv import load_dotenv
from intelli.flow import Agent, Task, Flow, TextTaskInput, AgentTypes

In [11]:
load_dotenv()


True

In [13]:
# Config
OPENAI_KEY = os.getenv("OPENAI_API_KEY")
MCP_URL = "http://localhost:8000/mcp"
OUTPUT_DIR = "./output"

Prepare the output folder and validate the keys

In [16]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"OpenAI key: {'✓' if OPENAI_KEY else '✗'}")

OpenAI key: ✓


## Utils

Common functions and preprocessors used throughout the code.

In [19]:
class MedicalDataProcessor:
    """Preprocessor to clean medical data and remove outcome leakage"""

    @staticmethod
    def remove_outcome_data(text_input):
        """Remove actual outcome columns from patient data using pandas"""
        if not text_input or not isinstance(text_input, str):
            return text_input
    
        try:
            import json
            data = json.loads(text_input.strip())
            
            if isinstance(data, list):
                df = pd.DataFrame(data)
            else:
                df = pd.DataFrame([data])
    
            outcome_columns = ['actualicumortality', 'actualiculos', 'expired']
            columns_to_drop = [col for col in outcome_columns if col in df.columns]
    
            if columns_to_drop:
                df = df.drop(columns=columns_to_drop)
                print(f"Preprocessor: Removed {len(columns_to_drop)} outcome columns: {columns_to_drop}")
    
            cleaned_text = df.to_csv(index=False)
            print(f"Preprocessor: Data shape after cleaning: {df.shape}")
    
            return cleaned_text
    
        except Exception as e:
            print(f"Preprocessor error: {e}, falling back to original data")
            return text_input

In [21]:
def extract_prediction_json(text_output):
    """Extract prediction JSON from model output"""
    try:
        json_pattern = r'\{[^{}]*"patient_id"[^{}]*\}'
        json_match = re.search(json_pattern, text_output, re.DOTALL)

        if json_match:
            return json.loads(json_match.group())

        return None
    except:
        return None

In [23]:
def extract_actual_outcome(raw_data, patient_id):
    """Extract actual outcome from raw MCP data using pandas"""
    try:
        if not raw_data or raw_data.strip() in ['[]', '[\n\n]', '']:
            print(f"MCP returned empty data for patient {patient_id}")
            return "UNKNOWN"
        
        data = json.loads(raw_data.strip())
        
        if isinstance(data, list):
            df = pd.DataFrame(data)
        else:
            df = pd.DataFrame([data])

        if df.empty:
            print(f"No data found for patient {patient_id}")
            return "UNKNOWN"
            
        patient_row = df[df['patientunitstayid'] == patient_id]
        
        if patient_row.empty:
            return "UNKNOWN"
        
        if 'actualicumortality' in patient_row.columns:
            mortality_value = patient_row['actualicumortality'].iloc[0]
            return "EXPIRED" if pd.notna(mortality_value) and "EXPIRED" in str(mortality_value).upper() else "SURVIVED"
        elif 'expired' in patient_row.columns:
            expired_value = patient_row['expired'].iloc[0]
            return "EXPIRED" if expired_value else "SURVIVED"
        
        return "UNKNOWN"
        
    except Exception as e:
        print(f"Error extracting outcome: {e}")
        return "UNKNOWN"

## Test MCP Server Connection
Verify that the MCP server is running and accessible.

In [26]:
# Create the MCP client agent
test_agent = Agent(
    agent_type=AgentTypes.MCP.value,
    provider="mcp",
    mission="Test server connection",
    model_params={"url": MCP_URL, "tool": "get_schema"}
)

In [28]:
async def test_mcp_connection():
    """Test MCP server connection using simple flow"""
    try:
        test_task = Task(TextTaskInput("Get server schema"), test_agent, log=False)
        test_flow = Flow(tasks={"test": test_task}, map_paths={"test": []}, log=False)
        
        result = await test_flow.start()
        schema_output = result["test"]["output"]
        
        if schema_output and "patientunitstayid" in schema_output:
            print("✓ MCP server connected successfully")
            return True
        else:
            print("✗ Patient ID column not found")  
            return False
            
    except Exception as e:
        print(f"✗ MCP connection failed: {e}")
        return False

In [30]:
connected = await test_mcp_connection()

Intelli MCPWrapper: Using streamablehttp_client for HTTP.
MCP Agent executing tool 'get_schema' with arguments: {}
Executing MCP tool 'get_schema' with arguments: {}
Filtered arguments for tool call: {}
Intelli MCPWrapper: Using full URL: http://localhost:8000/mcp for HTTP connection
Intelli MCPWrapper: Connecting to HTTP MCP server at http://localhost:8000/mcp
MCP tool execution result: meta=None content=[TextContent(type='text', text='{\n  "patientunitstayid": "int64",\n  "age": "int64",\n  "gender": "object",\n  "admissionheight": "float64",\n  "admissionweight": "float64",\n  "actualicumortality": "object",\n  "actualiculos": "float64",\n  "lab_count": "int64",\n  "has_wbc": "bool",\n  "has_creatinine": "bool",\n  "has_lactate": "bool",\n  "has_bilirubin": "bool",\n  "has_glucose": "bool",\n  "heartrate_mean": "float64",\n  "heartrate_max": "float64",\n  "systemicsystolic_mean": "float64",\n  "systemicsystolic_max": "float64",\n  "temperature_mean": "float64",\n  "temperature_max":

## Prediction Flow

Build a 2-agent flow to load patient data and predict mortality outcomes with preprocessing.


### Create Agents


In [34]:
PATIENT_IDS = [2834225, 1730330, 1591672, 1555349]

In [36]:
TEST_PATIENT_ID = PATIENT_IDS[0]

In [38]:
# Set up MCP data loading agent
data_agent = Agent(
    agent_type=AgentTypes.MCP.value,
    provider="mcp",
    mission="Load comprehensive patient clinical data",
    model_params={
        "url": MCP_URL,
        "tool": "filter_rows",
        "arg_column": "patientunitstayid",
        "arg_operator": "==",
        "arg_value": TEST_PATIENT_ID
    }
)

In [40]:
# prediction agent
prediction_agent = Agent(
    agent_type=AgentTypes.TEXT.value,
    provider="openai",
    mission="Predict patient mortality from clinical data",
    model_params={
        "key": OPENAI_KEY,
        "model": "gpt-4o",
        "max_tokens": 1000
    }
)

### Create Tasks

Define tasks with preprocessor to remove outcome data from prediction input.

In [43]:
data_task = Task(
    TextTaskInput("Load patient clinical data"),
    data_agent,
    log=True
)

Prediction task

In [46]:
prediction_prompt = """
Analyze the patient clinical data and predict mortality outcome.

Consider key clinical indicators:
- Demographics and severity scores
- Vital signs patterns
- Lab data and abnormalities
- APACHE predictions and scores

IMPORTANT: Return ONLY valid JSON in this exact format:
{
  "patient_id": PATIENT_ID_FROM_DATA,
  "prediction": "EXPIRED or SURVIVED",
  "key_factors": ["factor1", "factor2", "factor3"]
}

Do not include any text before or after the JSON. Only return the JSON object.
"""

In [48]:
prediction_task = Task(
    TextTaskInput(prediction_prompt),
    prediction_agent,
    pre_process=MedicalDataProcessor.remove_outcome_data,
    log=True
)

### Create Flow

Connect the MCP data loading agent with the prediction agent using preprocessor.

In [51]:
flow = Flow(
    tasks={
        "load_patient_data": data_task,
        "predict_mortality": prediction_task
    },
    map_paths={
        "load_patient_data": ["predict_mortality"],
    },
    log=True
)

#### Generate Flow Graph

In [54]:
flow.generate_graph_img( name="mcp_medical_flow", save_path=OUTPUT_DIR)

'./output/mcp_medical_flow.png'

#### Execute Flow

Run the prediction flow for the test patient. Change TEST_PATIENT_ID in previous cells

In [57]:
async def run_prediction_flow():
    """Run prediction flow for current TEST_PATIENT_ID"""

    print(f"\nRunning prediction for patient {TEST_PATIENT_ID}...")

    # Update data agent with current test patient ID
    data_agent.model_params["arg_value"] = TEST_PATIENT_ID

    try:
        # Run flow
        results = await flow.start()

        raw_data = results["load_patient_data"]["output"]
        prediction_output = results["predict_mortality"]["output"]

        print(f"Raw prediction output: {prediction_output[:150]}...")

        prediction_json = extract_prediction_json(prediction_output)
        actual_outcome = extract_actual_outcome(raw_data, TEST_PATIENT_ID)

        return prediction_json, actual_outcome

    except Exception as e:
        print(f"✗ Flow execution error: {e}")
        return None, None

In [59]:
prediction_result, actual_result = await run_prediction_flow()


Running prediction for patient 2834225...
---- Execute task load_patient_data (mcp/mcp) ----
MCP Agent executing tool 'filter_rows' with arguments: {'column': 'patientunitstayid', 'operator': '==', 'value': 2834225}
Executing MCP tool 'filter_rows' with arguments: {'column': 'patientunitstayid', 'operator': '==', 'value': 2834225}
Filtered arguments for tool call: {'column': 'patientunitstayid', 'operator': '==', 'value': 2834225}
Intelli MCPWrapper: Using full URL: http://localhost:8000/mcp for HTTP connection
Intelli MCPWrapper: Connecting to HTTP MCP server at http://localhost:8000/mcp
MCP tool execution result: meta=None content=[TextContent(type='text', text='[\n  {\n    "patientunitstayid":2834225,\n    "age":57,\n    "gender":"Female",\n    "admissionheight":165.0,\n    "admissionweight":95.0,\n    "actualicumortality":"ALIVE",\n    "actualiculos":9.727,\n    "lab_count":566,\n    "has_wbc":true,\n    "has_creatinine":true,\n    "has_lactate":true,\n    "has_bilirubin":true,\n 

Compare prediction to actual outcome and display results.

In [62]:
predicted = prediction_result.get("prediction", "UNKNOWN")
key_factors = prediction_result.get("key_factors", [])

In [64]:
print("\n" + "="*50)
print("PREDICTION RESULTS")
print("="*50)
print(f"Patient ID: {TEST_PATIENT_ID}")
print(f"Predicted: {predicted}")
print(f"Actual: {actual_result}")
print(f"Correct: {'✓' if predicted == actual_result else '✗'}")
print(f"Key factors: {', '.join(key_factors)}")
print("="*50)


PREDICTION RESULTS
Patient ID: 2834225
Predicted: EXPIRED
Actual: SURVIVED
Correct: ✗
Key factors: high heartrate_max, lab abnormalities, elevated APACHE predictions


# Appendix

## Get all patients

In [68]:
async def get_all_patient_ids_via_flow():
    """Get all patient IDs using MCP flow only"""
    try:
        # Create agent to get all data from server
        get_all_agent = Agent(
            agent_type=AgentTypes.MCP.value,
            provider="mcp",
            mission="Get all patient data",
            model_params={
                "url": MCP_URL,
                "tool": "get_head",
                "arg_n": 50  # Get enough rows to capture all patients
            }
        )
        
        # Create simple flow
        get_all_task = Task(TextTaskInput("Get all patients"), get_all_agent, log=False)
        get_all_flow = Flow(
            tasks={"get_all": get_all_task},
            map_paths={"get_all": []},
            log=False
        )
        
        # Execute flow
        result = await get_all_flow.start()
        raw_data = result["get_all"]["output"]
        
        # Parse JSON response and extract patient IDs
        
        data = json.loads(raw_data.strip())
        df = pd.DataFrame(data)
        
        # Get unique patient IDs
        patient_ids = df['patientunitstayid'].unique().tolist()
        print(f"Found {len(patient_ids)} patients via MCP flow")
        return patient_ids
        
    except Exception as e:
        print(f"Error getting patient IDs via flow: {e}")
        return [2834225, 1730330, 1591672, 1555349]

In [70]:
PATIENT_IDS = await get_all_patient_ids_via_flow()
PATIENT_IDS

MCP Agent executing tool 'get_head' with arguments: {'n': 50}
Executing MCP tool 'get_head' with arguments: {'n': 50}
Filtered arguments for tool call: {'n': 50}
Intelli MCPWrapper: Using full URL: http://localhost:8000/mcp for HTTP connection
Intelli MCPWrapper: Connecting to HTTP MCP server at http://localhost:8000/mcp
MCP tool execution result: meta=None content=[TextContent(type='text', text='[\n  {\n    "patientunitstayid":224053,\n    "age":86,\n    "gender":"Female",\n    "admissionheight":157.5,\n    "admissionweight":59.5,\n    "actualicumortality":"ALIVE",\n    "actualiculos":2.0486,\n    "lab_count":69,\n    "has_wbc":true,\n    "has_creatinine":true,\n    "has_lactate":false,\n    "has_bilirubin":false,\n    "has_glucose":true,\n    "heartrate_mean":72.29,\n    "heartrate_max":98.0,\n    "systemicsystolic_mean":151.98,\n    "systemicsystolic_max":203.0,\n    "temperature_mean":null,\n    "temperature_max":null,\n    "expired":false\n  },\n  {\n    "patientunitstayid":349887

[224053,
 349887,
 552515,
 702695,
 935285,
 976513,
 1555349,
 1591672,
 1730330,
 2834225]